In [1]:
!pip install -q transformers langchain-community httpx tqdm sentence-transformers pyvi underthesea faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 88.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 82.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 67.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 18.1 MB/s eta 0:00:00
   ━━━━━

# Preperation

Import the libraries

In [2]:
import httpx
from kaggle_secrets import UserSecretsClient
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document
import re
from typing import List
from bs4 import BeautifulSoup
import bs4
from IPython.display import display, Markdown
import requests
import datetime
from datetime import date
import tqdm
from sentence_transformers import SentenceTransformer
from pyvi.ViTokenizer import tokenize
from tqdm import tqdm 
import numpy as np
import faiss
from copy import deepcopy
from underthesea import sent_tokenize
import torch

In [3]:
API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
SEARCH_KEY = UserSecretsClient().get_secret("SEARCH_KEY")
# genai.configure(api_key=UserSecretsClient().get_secret("GEMINI_API_KEY"))

Helper functions

In [4]:
def preprocess_documents(documents: List[Document]) -> List[Document]:
    processed_docs = []
    for doc in documents:
        # Remove excessive whitespace and newlines
        text = re.sub(r'\s+', ' ', doc.page_content).strip()

        # Remove any remaining script or style tags if they were not already removed
        text = re.sub(r'<script.*?</script>', '', text, flags=re.DOTALL)
        text = re.sub(r'<style.*?</style>', '', text, flags=re.DOTALL)

        # Remove any leftover special characters and whitespace
        text = re.sub(r'[^\w\s.,?!()/;-]', '', text)
        text = re.sub(r'\s+', ' ', text).strip()

        # Preserve the metadata (source, title, etc.)
        processed_doc = Document(page_content=text, metadata=doc.metadata)
        processed_docs.append(processed_doc)

    return processed_docs

In [5]:
# Main function to search, extract URLs, and load documents
def search_google(query: str, topk: int = 3, lan: str = 'vi', **params):
    # Perform Google search
    base_url = 'https://www.googleapis.com/customsearch/v1'
    params = {
        'key': API_KEY,
        'cx': SEARCH_KEY,
        'q': query,
        'hl': lan,
        **params
    }
    response = httpx.get(base_url, params=params)
    response.raise_for_status()
    items = response.json().get('items', [])
    urls = [item['link'] for item in items]

    # Validate the URLs:
    unique_urls=set()
    search_urls=[]
    
    for url in urls:
        if url.split("/")[2] not in unique_urls:
            try:
                response = requests.get(url, timeout=1) 
                status_code = response.status_code
                
            except requests.exceptions.RequestException as e:              
                status_code = None
                
            if status_code == 200:
                unique_urls.add(url.split("/")[2])
                print(f"Currently searching the website: {url}")
                search_urls.append(url) 
                
            if len(unique_urls) >= topk:
                break
    
    if search_urls:
        # Load documents from the URLs
        loader = WebBaseLoader(
            web_paths=search_urls,  # Limit to top 3 URLs
            bs_kwargs=dict(parse_only=bs4.SoupStrainer()),
            requests_kwargs={"timeout": 600}
        )
        docs = loader.load()
        return preprocess_documents(docs)

        # # Create a formatted string of document data
        # doc_string = "\n\n".join(
        #     f"URL {index + 1}\n"
        #     f"Source: {doc.metadata.get('source', 'N/A')}\n"
        #     f"Title: {doc.metadata.get('title', 'N/A')}\n"
        #     f"Description: {doc.metadata.get('description', 'N/A')}\n"
        #     f"Content: {doc.page_content}\n"
        #     for index, doc in enumerate(docs)
        # )

        # return doc_string

In [6]:
text_splitter = CharacterTextSplitter(
    chunk_size=2000, 
    chunk_overlap=0,
    separator=" ",
    length_function=len
)

docs = search_google('pickleball là gì', 3, 'vi')
texts = text_splitter.split_documents(docs)

Currently searching the website: https://vi.wikipedia.org/wiki/Pickleball
Currently searching the website: https://nhathuoclongchau.com.vn/bai-viet/pickleball-la-gi-luat-choi-va-ky-thuat-co-ban-ban-can-nam.html
Currently searching the website: https://irace.vn/pickleball-la-gi/


In [7]:
def splits_to_dicts(splitted_docs):
    doc_dicts = []
    for idx, doc in enumerate(splitted_docs):
        doc_dict = {
            "source": doc.metadata.get("source", "unknown"),
            "title": doc.metadata.get("title", "No Title"),
            "passage": doc.page_content,
            "id": idx
        }
        doc_dicts.append(doc_dict)
    return doc_dicts

corpus = splits_to_dicts(texts)

# Retrieval


Get the Vietnamese embedding

In [8]:
emb_model = SentenceTransformer('bkai-foundation-models/vietnamese-bi-encoder').cuda()

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

In [9]:
def retrieve(query, corpus=corpus,topk=10):
    segmented_corpus = [tokenize(example['passage']) for example in tqdm(corpus)]
    embeddings = emb_model.encode(segmented_corpus, convert_to_numpy=True)
    embeddings /= np.linalg.norm(embeddings, axis=1)[:, np.newaxis]
    
    # Create a FAISS index
    embedding_dim = embeddings.shape[1] 
    index = faiss.IndexFlatIP(embedding_dim)  
    faiss.normalize_L2(embeddings) 
    index.add(embeddings)  
    
    # Perform a search
    query_embedding = emb_model.encode([tokenize(query)], convert_to_numpy=True)
    query_embedding /= np.linalg.norm(query_embedding, axis=1, keepdims=True)
    faiss.normalize_L2(query_embedding)    
    
    distances, indices = index.search(query_embedding, topk)    
    results = []
    for i, idx in enumerate(indices[0]):
        result = deepcopy(corpus[idx]) 
        result["score"] = float(distances[0][i]) 
        results.append(result)

    return results    

Some heuristic functions to improve the retrieval

In [10]:
def extract_consecutive_subarray(numbers):
    subarrays = []
    current_subarray = []
    for num in numbers:
        if not current_subarray or num == current_subarray[-1] + 1:
            current_subarray.append(num)
        else:
            subarrays.append(current_subarray)
            current_subarray = [num]

    subarrays.append(current_subarray)  # Append the last subarray
    return subarrays

def merge_contexts(passages):
    passages_sorted_by_id = sorted(passages, key=lambda x: x["id"])

    psg_ids = [x["id"] for x in passages_sorted_by_id]
    consecutive_ids = extract_consecutive_subarray(psg_ids)

    merged_contexts = []
    b = 0
    for ids in consecutive_ids:
        psgs = passages_sorted_by_id[b:b+len(ids)]
        
        # Group passages by title within the consecutive IDs
        title_groups = {}
        for psg in psgs:
            title = psg["title"]
            if title not in title_groups:
                title_groups[title] = []
            title_groups[title].append(psg)
        
        # Merge passages in each title group
        for title, group in title_groups.items():
            if len(group) == 1:
                # If only one passage in the group, add it as is
                psg = group[0]
                merged_contexts.append(dict(
                    title=psg['title'],
                    passage=psg['passage'],
                    score=psg["score"],
                    merged_from_ids=[psg["id"]]
                ))
            else:
                # Merge passages with the same 
                psg_texts = [x["passage"] for x in group]
                merged = " ".join(psg_texts)
                merged_contexts.append(dict(
                    title=group[0]['title'],
                    passage=merged,
                    score=max([x["score"] for x in group]),
                    merged_from_ids=[x["id"] for x in group]
                ))

        b = b + len(ids)

    return merged_contexts

def discard_contexts(passages):
    sorted_passages = sorted(passages, key=lambda x: x["score"])
    if len(sorted_passages) == 1:
        return sorted_passages
    else:
        shortened = deepcopy(sorted_passages)
        for i in range(len(sorted_passages) - 1):
            current, next = sorted_passages[i], sorted_passages[i+1]
            if next["score"] - current["score"] >= 0.05:
                shortened = sorted_passages[i+1:]
        return shortened

def expand_context(passage, corpus=corpus, word_window=60, n_sent=3):
    # psg_id = passage["id"]
    merged_from_ids = passage["merged_from_ids"]
    title = passage["title"]
    prev_id = merged_from_ids[0] - 1
    next_id = merged_from_ids[-1] + 1

    texts = []
    if prev_id in range(0, len(corpus)):
        prev_psg = corpus[prev_id]
        if prev_psg["title"] == title:
            prev_text = prev_psg["passage"]
            # prev_text = " ".join(prev_text.split()[-word_window:])
            prev_text = " ".join(sent_tokenize(prev_text)[-n_sent:])
            texts.append(prev_text)

    texts.append(passage["passage"])

    if next_id in range(0, len(corpus)):
        next_psg = corpus[next_id]
        if next_psg["title"] == title:
            next_text = next_psg["passage"]
            # next_text = " ".join(next_text.split()[:word_window])
            next_text = " ".join(sent_tokenize(next_text)[:n_sent])
            texts.append(next_text)

    expanded_text = " ".join(texts)
    new_passage = deepcopy(passage)
    new_passage["passage"] = expanded_text
    return new_passage

def expand_contexts(passages, corpus=corpus, word_window=60, n_sent=3):
    new_passages = [expand_context(passage, corpus=corpus) for passage in passages]
    return new_passages

def collapse(passages):
    new_passages = deepcopy(passages)
    titles = {}
    for passage in new_passages:
        title = passage["title"]
        if not titles.get(title):
            titles[title] = [passage]
        else:
            titles[title].append(passage)
    best_passages = []
    for k, v in titles.items():
        best_passage = max(v, key= lambda x: x["score"])
        best_passages.append(best_passage)
    sorted_best_passages = sorted(best_passages, key=lambda x: x["score"], reverse=True)
    return sorted_best_passages

In [11]:
def smooth_contexts(passages, corpus=corpus):
    """Make the context fed to the LLM better.
    Args:
        passages (list): Chunks retrieved from BM25 + semantic retrieval.

    Returns:
        list: List of whole paragraphs, usually will be more relevant to the initital question.
    """
    # 1. If consecutive chunks are retrieved, merge them into one big chunk to ensure the continuity.
    merged_contexts = merge_contexts(passages)
    # 2. A heuristic to discard irrelevevant contexts.
    # It seems to be better to only keep what are elevant so that the model can focus.
    # Also this reduce #tokens LLM has to read.
    shortlisted_contexts = discard_contexts(merged_contexts)
    # 3. Another heuristic. this step is to take advantage of long context understanding of the LLM.
    # In many cases, the retrieved passages are just consecutive words, not a comprehensive paragraph.
    # This is to expand the passage to the whole paragraph that surrounds it.
    # My intuition about this is that whole paragraph will add necessary and relevant information.
    expanded_contexts = expand_contexts(shortlisted_contexts, corpus=corpus)
    # 4. Now after all the merging and expanding, if what are left for us is more than one paragraphs
    # from the same URL, then we will only take paragraph with highest retrieval score.
    collapsed_contexts = collapse(expanded_contexts)
    return collapsed_contexts

In [12]:
top_passages = retrieve('pickleball là gì')

100%|██████████| 18/18 [00:00<00:00, 202.28it/s]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
smooth_contexts(top_passages)

[{'title': 'Pickleball là gì? Nguồn gốc, cách chơi và những điều cần biết',
  'passage': 'Pickleball là gì? Nguồn gốc, cách chơi và những điều cần biết Facebook Instagram Mua vé sự kiện Virtual Race Sự Kiện tại Việt Nam Dịch vụ Bán Vé Sự Kiện Tìm Hình Sự Kiện Tổ Chức Sự Kiện Kiếm Tiền cùng iRace.vn Sản Xuất Vật Phẩm Huy chương Quần áo, Phụ kiện Thông Tin Về iRace Báo Chí Viết Hỏi Đáp Hướng Dẫn Sử Dụng Kiến Thức Chạy Bộ Shop Thể Thao Dinh Dưỡng Phụ kiện Đăng nhập Tiếng Việt TÌM KIẾM Đăng nhập Đăng nhập tài khoản Tài khoản mật khẩu của bạn Forgot your password? Get help Điều khoản sử dụng Khôi phục mật khẩu Khởi tạo mật khẩu email của bạn Mật khẩu đã được gửi vào email của bạn. iRace.vn Nền tảng tổ chức chạy bộ trực tuyến 1 Việt Nam Mua vé sự kiện Virtual Race Sự Kiện tại Việt Nam Dịch vụ Bán Vé Sự Kiện Tìm Hình Sự Kiện Tổ Chức Sự Kiện Kiếm Tiền cùng iRace.vn Sản Xuất Vật Phẩm Huy chương Quần áo, Phụ kiện Thông Tin Về iRace Báo Chí Viết Hỏi Đáp Hướng Dẫn Sử Dụng Kiến Thức Chạy Bộ Shop Th

# Generation

Import the generative model

In [14]:
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer
LLM_model_path = "vinai/PhoGPT-4B-Chat"  

config = AutoConfig.from_pretrained(LLM_model_path, trust_remote_code=True)

LLM_model = AutoModelForCausalLM.from_pretrained(LLM_model_path, config=config, torch_dtype=torch.bfloat16, trust_remote_code=True)
LLM_model.to("cuda")
LLM_model.eval()  

tokenizer = AutoTokenizer.from_pretrained(LLM_model_path, trust_remote_code=True)  

PROMPT_TEMPLATE = "### Câu hỏi: {instruction}\n### Trả lời:"  

# Some instruction examples
# instruction = "Viết bài văn nghị luận xã hội về {topic}"
# instruction = "Viết bản mô tả công việc cho vị trí {job_title}"
# instruction = "Sửa lỗi chính tả:\n{sentence_or_paragraph}"
# instruction = "Dựa vào văn bản sau đây:\n{text}\nHãy trả lời câu hỏi: {question}"
# instruction = "Tóm tắt văn bản:\n{text}"
# instruction = "Sửa lỗi chính tả:\nTriệt phá băng nhóm kướp ô tô, sử dụng \"vũ khí nóng\""

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

configuration_mpt.py:   0%|          | 0.00/16.4k [00:00<?, ?B/s]

attention.py:   0%|          | 0.00/24.6k [00:00<?, ?B/s]

flash_attn_triton.py:   0%|          | 0.00/28.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vinai/PhoGPT-4B-Chat:
- flash_attn_triton.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

pytorch_model.bin:   0%|          | 0.00/7.38G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/91.0 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/850 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/844k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

RAG

In [15]:
def chat_RAG(search_topk=3, rag_topk=10):
    search_query = input("Please input your search query. Enter q to quit >")
    if search_query == "q":
        pass
    docs = search_google(search_query, search_topk)
    text_splitter = CharacterTextSplitter(
    chunk_size=2000, 
    chunk_overlap=0,
    separator=" ",
    length_function=len
)
    texts = text_splitter.split_documents(docs)
    corpus = splits_to_dicts(texts)
    while True:
        chat_query = input("Please input your query. Enter q to quit >")
        if input == "q":
            return
        top_passages = retrieve(chat_query, corpus, rag_topk)
        contexts = smooth_contexts(top_passages, corpus)
        instruction = "Dựa vào văn bản sau đây:\n{text}\nHãy trả lời câu hỏi: {question}".format_map({"text": contexts, "question": chat_query})
        input_prompt = PROMPT_TEMPLATE.format_map({"instruction": instruction}) 
        input_ids = tokenizer(input_prompt, return_tensors="pt")
        outputs = LLM_model.generate(  
        inputs=input_ids["input_ids"].to("cuda"),  
        attention_mask=input_ids["attention_mask"].to("cuda"),  
        do_sample=True,  
        temperature=1.0,  
        top_k=50,  
        top_p=0.9,  
        max_new_tokens=1024,  
        eos_token_id=tokenizer.eos_token_id,  
        pad_token_id=tokenizer.pad_token_id)  
        response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]  
        response = response.split("### Trả lời:")[1]
        print(response)
        print("---" * 30)

# Final Result

In [16]:
chat_RAG()

Please input your search query. Enter q to quit > đại học Bách Khoa Hà Nội


Currently searching the website: https://hust.edu.vn/
Currently searching the website: https://vi.wikipedia.org/wiki/%C4%90%E1%BA%A1i_h%E1%BB%8Dc_B%C3%A1ch_khoa_H%C3%A0_N%E1%BB%99i
Currently searching the website: https://xaydungchinhsach.chinhphu.vn/diem-chuan-dai-hoc-bach-khoa-ha-noi-11924081717400873.htm


Please input your query. Enter q to quit > đại học Bách Khoa Hà Nội được thành lập năm nào, do ai


100%|██████████| 29/29 [00:00<00:00, 185.95it/s]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Please input your query. Enter q to quit > q


100%|██████████| 29/29 [00:00<00:00, 172.76it/s]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 